In [1]:
import os

from docarray import Document, DocumentArray
from fastapi import APIRouter
from tqdm import tqdm

from routers.processor_service import ProcessorService
from routers.processor_router import preprocess_documents

from gensim import corpora, models

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import gensim
import nltk

import spacy

import re

In [2]:
import re

import PyPDF2
import spacy
from dateutil import parser
from docarray import DocumentArray

import logging

nlp = spacy.load("pt_core_news_sm")


In [3]:
    def extract_text(doc_array) -> str:
        text = ""
        for document in doc_array:
            pdf_path = document.tags.get("path")
            if pdf_path:
                read_pdf = PyPDF2.PdfReader(pdf_path)
                number_of_pages = len(read_pdf.pages)
                for page_num in range(number_of_pages):
                    page = read_pdf.pages[page_num]
                    content = page.extract_text()
                    doc = nlp(content)
                    text += doc.text + "\n"

        return text

In [11]:
import pprint
root_directory = "documents_test"
results = []
qtd_arquivos = 0
for year in range(2004, 2015):
    for folder_name in ["Consuni", "Consepe"]:
            folder_path = os.path.join(root_directory, folder_name)

            for root, dirs, files in os.walk(folder_path):
                for filename in files:
                    pdf_path = os.path.join(root, filename)
                    results.append(pdf_path)
                    print(pdf_path)
                    doc_array = DocumentArray()
                    doc_array.append(Document(path=pdf_path))
                    extracted_text = extract_text(doc_array)
                    results.append(extracted_text)



documents_test\Consuni\2004\01-2004 - Lista tríplice 2004.pdf
documents_test\Consuni\2004\02-2004 - Assembléia Constituinte.pdf
documents_test\Consuni\2004\03-2004 - Regimento CONSUNI.pdf
documents_test\Consuni\2004\09-2005 - Regimento Interno Tocantinópolis (Alterado pela Resolução Consuni nº 13-2009).pdf
documents_test\Consuni\2004\36-2021 - Calendário Acadêmico UFT 2021.1 (Recuperação Automática).pdf
documents_test\Consuni\2004\51-2021 - Calendário Acadêmico UFT 2022 - Consuni-UFT.pdf
documents_test\Consuni\2004\01-2004 - Lista tríplice 2004.pdf
documents_test\Consuni\2004\02-2004 - Assembléia Constituinte.pdf
documents_test\Consuni\2004\03-2004 - Regimento CONSUNI.pdf
documents_test\Consuni\2004\09-2005 - Regimento Interno Tocantinópolis (Alterado pela Resolução Consuni nº 13-2009).pdf
documents_test\Consuni\2004\36-2021 - Calendário Acadêmico UFT 2021.1 (Recuperação Automática).pdf
documents_test\Consuni\2004\51-2021 - Calendário Acadêmico UFT 2022 - Consuni-UFT.pdf
documents_test

In [70]:
import pprint
def preprocess_text(text) -> str:

        date_pattern_2 = re.compile(r'\b\d{4}-\d{1,2}\b')

        doc = nlp(text)
        text_formatted = spacy.tokens.Doc(
            doc.vocab,
            words=[
                token.text.lower()
                for token in doc
                if not token.is_space
                and not token.is_punct
                and not token.like_num
                and not token.is_bracket
                and not date_pattern_2.match(token.text)
                # and not pontuation_pattern.match(token.text)
          
            ],
        )


        text_formatted = " ".join([token.text for token in text_formatted if not token.is_space and not token.is_bracket and not date_pattern_2.match(token.text) and not token.is_punct and not token.like_num ])
        return text_formatted

# pprint.pprint(preprocess_text(results[0]))
# pprint.pprint(preprocess_text(results[1]))
pprint.pprint(preprocess_text(results[1]))


('serviço público federal fundação universidade federal do tocantins resolução '
 'do conselho universitário nº regulamenta o processo de escolha das listas '
 'tríplices para reitor e vice-reitor da universidade federal do tocantins uft '
 'mandato o conselho universitário em sessão realizada no dia 14/04/2004 tendo '
 'em vista a lei nº de de dezembro de e o decreto nº de de maio de que '
 'normatizam o processo para escolha das listas tríplices para reitor e vice- '
 'reitor resolve do colégio eleitoral art. o colégio eleitoral que indicará as '
 'lis tas tríplices para reitor e vice- reitor da universidade federal do '
 'tocantins uft mandato 2004-2008 será constituído pelos membros do conselho '
 'universitário parágrafo único o colégio eleitoral será presid ido pelo '
 'reitor da uft dos candidatos art. somente poderão compor as listas trípli '
 'ces docentes integrantes da carreira de magistério superior ocupantes dos '
 'cargo s de professor titular de professor adjunto ou que 

In [72]:

def remove_unecessary_data(text):
    texto = preprocess_text(text)
    unnecessary = [
        "universidade",
        "federal",
        "tocantins",
        "fundação"
        "fundação universidade federal tocantins",
        "uft",
        "universidad federal tocantins",
        "art",
        ".",
        "°",
        " universidade federal tocantins"
    ]
    words = texto.split()
    filtered_words = [word for word in words if word.lower() not in unnecessary]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

texto = remove_unecessary_data(results[1])

print(texto)



serviço público fundação do resolução do conselho universitário nº regulamenta o processo de escolha das listas tríplices para reitor e vice-reitor da do mandato o conselho universitário em sessão realizada no dia 14/04/2004 tendo em vista a lei nº de de dezembro de e o decreto nº de de maio de que normatizam o processo para escolha das listas tríplices para reitor e vice- reitor resolve do colégio eleitoral art. o colégio eleitoral que indicará as lis tas tríplices para reitor e vice- reitor da do mandato 2004-2008 será constituído pelos membros do conselho universitário parágrafo único o colégio eleitoral será presid ido pelo reitor da dos candidatos art. somente poderão compor as listas trípli ces docentes integrantes da carreira de magistério superior ocupantes dos cargo s de professor titular de professor adjunto ou que sejam portadores do título de douto r conforme estabelece a legislação em vigor do processo art. o voto é secreto e só poderá ser efetua do pelos membros do colégi

In [73]:
import spacy

doc = nlp(texto)


# Encontrar padrões de ação e objeto
for token in doc:
    # Verificar se o token é um verbo relacionado à ação
    if token.pos_ == "VERB" and token.text.lower() in ["regulamenta", "dispõe", "cria", "estabelece", 
                                                       "institui", "aprova", "altera", "anula", 
             
                                                       "convoca", "instituir"]:
        # Encontrar o objeto direto do verbo
        objeto_direto = [child.text for child in token.children if child.dep_ == "obj"]
        # encontrar proximos verbos e objetos
        
        # Imprimir a informação
        if objeto_direto:
            
            pretty(f"A resolução 01-2004 -> {token.text} -> {objeto_direto[0]}.")


A resolução 01-2004 -> regulamenta -> processo.


In [62]:
verbos_interesse = ["regulamenta", "dispõe", "cria", "estabelece", 
                    "institui", "aprova", "altera", "anula", 
                    "convoca", "instituir"]

import pprint
from spacy import displacy

for token in doc:
    
    # Verificar se o token é um verbo de interesse
    if token.pos_ == "VERB" and token.text.lower() in verbos_interesse:
        verb = token.text
        indice_verbo = token.i
        indice_final_sentenca = token.sent[-1].i
       

        # Definir um limite para a quantidade de palavras após o verbo
        limite_palavras_apos_verbo = 12

        # Pegar os índices dos tokens imediatamente após o verbo, até o limite definido
        indices_palavras_apos_verbo = [i + 1 for i in range(indice_verbo, indice_final_sentenca) if i + 1 <= indice_final_sentenca][:limite_palavras_apos_verbo]

        tokens_entre_verbo_e_palavras = [doc[i] for i in range(indice_verbo, max(indices_palavras_apos_verbo))]
        
        texto_entre_verbo_e_palavras = ' '.join(token.text for token in tokens_entre_verbo_e_palavras)

        print(texto_entre_verbo_e_palavras)



dispõe sobre as normas de funcionamento da assembléia constituinte interna da do
estabelece o conselho nacional de educação art. o funcionamento da assembléia constituinte


In [59]:
import os
from PIL import Image

# Specify the output file path
output_path = os.path.join("path", "to", "output", "image.png")

string = "regulamenta processo escolha listas tríplices reitor vice-reitor"
doc = nlp(string)
displacy.render(doc, style="dep", jupyter=False, options={"compact": False, "color": "#40398F"})
# Salvar a imagem gerada
output_path = os.path.join("", "imagaae.png")
image_path = output_path.replace(".png", "_dep.png")  # Adicione um sufixo para diferenciar do texto
plt.savefig(image_path, format="png", bbox_inches="tight", pad_inches=0.1)
plt.close()  # Feche a figura para liberar recursos



In [80]:
string1 = "dispõe sobre o calendário acadêmico da do para o ano letivo de o egrégio conselho universitário consuni da do reunido"

string2 = "dispõe sobre o calendário acadêmico o ano letivo de o egrégio conselho universitário consuni da do reunido"


subject_doc = nlp(string1)
object_doc = nlp(string2)

similarity_score = subject_doc.similarity(object_doc)

print("Similarity Score:", similarity_score)
print("=" * 50)

Similarity Score: 0.970405028733867


C:\Users\Aline\AppData\Local\Temp\ipykernel_16756\3822220296.py:9: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  similarity_score = subject_doc.similarity(object_doc)


In [81]:
verbos_interesse = ["regulamenta", "dispõe", "cria", "estabelece", 
                    "institui", "aprova", "altera", "anula", 
                    "convoca", "instituir"]

import pprint

doc_texto2 = nlp(texto2)

for token in doc_texto2:
    # Verificar se o token é um verbo de interesse
    if token.pos_ == "VERB" and token.text.lower() in verbos_interesse:

        indice_verbo2 = token.i
        indice_final_sentenca2 = token.sent[-1].i
        print(indice_verbo2)
        print(indice_final_sentenca2)

        # Definir um limite para a quantidade de palavras após o verbo
        limite_palavras_apos_verbo2 = 20

        # Pegar os índices dos tokens imediatamente após o verbo, até o limite definido
        indices_palavras_apos_verbo2 = [i + 1 for i in range(indice_verbo2, indice_final_sentenca2) if i + 1 <= indice_final_sentenca2][:limite_palavras_apos_verbo2]
        
        print(indices_palavras_apos_verbo2)
        # Ensure that indices_palavras_apos_verbo2 does not exceed the length of the document
        #indices_palavras_apos_verbo2 = [i for i in indices_palavras_apos_verbo2 if i < len(doc_texto2)]
        
        tokens_entre_verbo_e_palavras2 = [doc_texto2[i] for i in indices_palavras_apos_verbo2]
  
        texto_entre_verbo_e_palavras2 = ' '.join(token.text for token in tokens_entre_verbo_e_palavras2)

        print(texto_entre_verbo_e_palavras2)
        # doc = nlp(texto_entre_verbo_e_palavras)
        # print(type(doc))
        
        # displacy.render(texto_entre_verbo_e_palavras, style="dep")
            

29
849
[30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
sobre o calendário acadêmico da do para o ano letivo de o egrégio conselho universitário consuni da do reunido em


In [82]:
from spacy import displacy

import pprint
texto_entre_verbo_e_palavras = ' '.join(token.text for token in tokens_entre_verbo_e_palavras)

doc_tex = nlp(texto_entre_verbo_e_palavras)
lista = ""
for token in doc:
   # print(f"Token: {token.text} POS: {token.pos_} Dep: {token.dep_}, Entity Type: {token.ent_type_}")
    #print(f"Token: {token.text} POS: {token.pos_} Dep: {token.dep_}")
    if token.pos_ == "VERB" and token.text.lower() in ["regulamenta", "dispõe", "cria", "estabelece", 
                                                       "institui", "aprova", "altera", "anula", 
             
                                                       "convoca", "instituir"]:
        verb = token.text
    # objetos_adverbos = [child for child in token]
    
    for child in token.children:
  
        if child.dep_ == "obj":
            lista += child.text + " "
        if child.dep_ == "obl":
            lista += child.text + " "
        if child.dep_ == "nsubj":
            lista += child.text + " "
        if child.dep_ == "nmod":
            print(child.text, 'nmod')
            lista += child.text + " "
        if child.dep_ == "conj":
            print(child.text, 'conj')
            lista += child.text + " "
        if child.dep_ == "amod":
            print(child.text, 'amod')
            lista += child.text + " "
 
# print(lista)
all_of_them = verb + " " + lista
print(f"A resolução {all_of_them}")


        # encontrar proximos verbos e objetos
    # print(objetos_adverbos)
    # print(f"A resolução X {token.text} {objetos_adverbos} ")

acadêmico amod
da nmod
ano nmod
letivo amod
egrégio nmod
consuni nmod
universitário amod
da nmod
reunido nmod
A resolução dispõe calendário conselho acadêmico da ano letivo egrégio consuni universitário da reunido 


In [83]:
from spacy import displacy

import pprint

texto_entre_verbo_e_palavras2 = ' '.join(token.text for token in tokens_entre_verbo_e_palavras2)

doc_tex = nlp(texto_entre_verbo_e_palavras2)
pprint.pprint(doc_tex)
lista = ""
for token in doc_tex:
   # print(f"Token: {token.text} POS: {token.pos_} Dep: {token.dep_}, Entity Type: {token.ent_type_}")
    #print(f"Token: {token.text} POS: {token.pos_} Dep: {token.dep_}")
    if token.pos_ == "VERB" and token.text.lower() in ["regulamenta", "dispõe", "cria", "estabelece", 
                                                       "institui", "aprova", "altera", "anula", 
             
                                                       "convoca", "instituir"]:
        verb = token.text
    # objetos_adverbos = [child for child in token]
    
    for child in token.children:
  
        if child.dep_ == "obj":
            lista += child.text + " "
        if child.dep_ == "obl":
            lista += child.text + " "
        if child.dep_ == "nsubj":
            lista += child.text + " "
        if child.dep_ == "nmod":
            print(child.text, 'nmod')
            lista += child.text + " "
        if child.dep_ == "conj":
            print(child.text, 'conj')
            lista += child.text + " "
        if child.dep_ == "amod":
            print(child.text, 'amod')
            lista += child.text + " "
 
# print(lista)
all_of_them = verb + " " + lista
print(f"A resolução {all_of_them}")


        # encontrar proximos verbos e objetos
    # print(objetos_adverbos)
    # print(f"A resolução X {token.text} {objetos_adverbos} ")

sobre o calendário acadêmico da do para o ano letivo de o egrégio conselho universitário consuni da do reunido em
acadêmico amod
da nmod
ano nmod
letivo amod
egrégio nmod
consuni nmod
universitário amod
da nmod
reunido nmod
A resolução dispõe acadêmico da ano letivo egrégio consuni universitário da reunido 
